In [19]:
# Modelo LSTM
# python3 -m venv env
# source env/bin/activate
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html


In [20]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import joblib
from tensorflow.keras.losses import MeanSquaredError
import tensorflow as tf


In [21]:
# Dados para previsão
new_data = np.array([
    [51, 49, 1.04],
    [56, 53, 1.05],
    [62, 61, 1.01],
    [60, 65, 0.92],
    [69, 70, 0.98],
    [58, 55, 1.05],
    [68, 70, 0.97]
])

# Carregar o escalador salvo
scaler = joblib.load('scaler.joblib')

# Escalar os novos dados
new_data_scaled = scaler.transform(new_data)

# Transformar os dados para a forma [samples, time steps, features]
time_step = 2
X_new = []

for i in range(len(new_data_scaled) - time_step + 1):
    X_new.append(new_data_scaled[i:(i + time_step), :])

X_new = np.array(X_new)
print(f'X_new shape: {X_new.shape}')  # Esperado: (n_samples, time_steps, n_features)

# Carregar o modelo treinado no novo formato .keras
model = load_model('modelo_treinado.keras')

# Compilar o modelo novamente após carregar
model.compile(optimizer='adam', loss=MeanSquaredError())

# Definindo a função de previsão fora do loop
# time_step = Numero de time steps, dexiar igual ao software de abstração
# 3 = Numero de features (colunas do dataset / array)
@tf.function(input_signature=[tf.TensorSpec(shape=[None, time_step, 3], dtype=tf.float32)])
def predict_function(input_data):
    return model(input_data, training=False)

# Fazer previsões
predictions = predict_function(tf.convert_to_tensor(X_new, dtype=tf.float32))

# Inverter a escala das previsões
predictions_inversed = scaler.inverse_transform(predictions.numpy())

# Exibir previsões
print("Previsões:")
print(predictions_inversed)

X_new shape: (6, 2, 3)


Previsões:
[[43.76998    48.52135     0.8893254 ]
 [45.51436    49.80919     0.8998455 ]
 [47.704144   51.748486    0.9077573 ]
 [49.509468   53.375256    0.91485447]
 [48.93977    52.498466    0.91777813]
 [46.935852   50.955116    0.9072458 ]]
